# Library

In [ ]:
!pip install ragas google-search-results wolframalpha

In [23]:
from src.utils import show_stream, naver_news_crawler

from dotenv import load_dotenv

import os
import numpy as np
import pickle
import json
import bs4
import itertools
from bs4 import BeautifulSoup
import requests
import uuid
import nest_asyncio
import huggingface_hub
from tqdm import tqdm
from operator import itemgetter
from datetime import datetime
from pydantic import BaseModel, Field
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from seaborn import load_dataset
from datasets import Dataset

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_core.load import dumpd, dumps, load, loads
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.callbacks.manager import get_openai_callback
from langchain_community.chat_models import ChatOllama
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
    MaxMarginalRelevanceExampleSelector,
)
from langchain_core.output_parsers import (
    StrOutputParser,
    PydanticOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser,
)
from langchain.output_parsers.structured import (
    ResponseSchema,
    StructuredOutputParser,
)
from langchain.output_parsers.pandas_dataframe import PandasDataFrameOutputParser
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from langchain.output_parsers.retry import RetryOutputParser
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda,
    ConfigurableField,
)

from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import (
    UnstructuredPDFLoader,
    PyPDFium2Loader,
    PDFMinerLoader,
    PyPDFDirectoryLoader,
    PDFPlumberLoader,
    UnstructuredExcelLoader,
    DataFrameLoader,
    Docx2txtLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredPowerPointLoader,
    WebBaseLoader,
    TextLoader,
    DirectoryLoader,
    JSONLoader,
    ArxivLoader,
)

from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    Language,
    MarkdownHeaderTextSplitter,
    HTMLHeaderTextSplitter,
    RecursiveJsonSplitter,
)
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.storage import (
    LocalFileStore,
    InMemoryByteStore,
    InMemoryStore,
)
from langchain.embeddings import CacheBackedEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_chroma import Chroma

from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import (
    ContextualCompressionRetriever,
    BM25Retriever,
    EnsembleRetriever,
    ParentDocumentRetriever,
    TimeWeightedVectorStoreRetriever,
)
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

from langchain.chains import (
    LLMChain,
    SimpleSequentialChain,
    SequentialChain,
    RetrievalQA,
    RetrievalQAWithSourcesChain,
)
from langchain.chains.summarize import load_summarize_chain
from langchain.agents import load_tools, initialize_agent
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.memory import VectorStoreRetrieverMemory

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from ragas.testset.transforms import default_transforms
from ragas.testset.synthesizers import (
    default_query_distribution,
    AbstractQuerySynthesizer,
    ComparativeAbstractQuerySynthesizer,
    SpecificQuerySynthesizer,
)
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate

In [2]:
load_dotenv()
set_llm_cache(InMemoryCache())

In [3]:
huggingface_hub.login()

# LangChain

자연어 처리와 대화형 AI 애플리케이션을 구축하는 데 도움을 주는 프레임워크. <br>
여러 가지 NLP 모델과 데이터 소스를 쉽게 통합할 수 있도록 설계. <br>
복잡한 대화형 시스템을 개발하는 데 필요한 다양한 도구와 컴포넌트를 제공. <br>
대화형 애플리케이션, 챗봇, 자동화된 고객 서비스 솔루션 등을 보다 효율적으로 개발 가능. <br>

<br>

<font style="font-size:20px"> 특징 </font>

- 모델 통합: 다양한 언어 모델(예: GPT, Claude 등)을 쉽게 사용할 수 있도록 지원
- 데이터 소스 연결: 데이터베이스, API, 파일 시스템 등 여러 소스와 연결할 수 있는 기능 제공
- 체인 구성: 여러 작업을 순차적으로 연결하여 복잡한 프로세스를 만들 수 있는 체인 개념 도입
- 사용자 정의: 사용자 요구에 맞게 쉽게 확장하고 커스터마이즈 가능

## RAG

Retrieval-Augmented Generation(RAG)는 정보 검색과 생성을 통합하는 방법론. <br>
RAG는 대규모 문서 데이터베이스에서 관련 정보를 검색하고, 이를 통해 모델이 더 정확하고 상세한 답변을 생성할 수 있도록 함. <br>
최신 뉴스 이벤트나 특정 분야의 전문 지식과 같은 주제에 대해 물어보면, RAG는 관련 문서를 찾아 그 내용을 바탕으로 답변.

|사전 준비 단계|Runtime 단계|
|-----------|-----------|
|![](https://wikidocs.net/images/page/233780/rag-graphic-1.png)|![](https://wikidocs.net/images/page/233780/rag-graphic-2.png)|
|Document Load: 외부 데이터 소스에서 필요한 문서를 로드|Retriever: 질문이 주어지면 관련된 내용을 벡터 데이터베이스에서 검색|
|Text Splitter: 로드된 문서를 처리 가능한 작은 단위로 분할|Prompt: 검색된 정보를 바탕으로 언어 모델을 위한 질문 구성|
|Embedding: 각 문서 또는 문서의 일부를 벡터 형태로 변환|LLM: 구성된 프롬프트를 사용하여 언어 모델이 답변 생성|
|Vector Store: 임베딩된 벡터들을 데이터베이스에 저장. 요약된 키워드를 색인화하여 나중에 빠르게 찾을 수 있도록 함|Chain: 이전의 모든 과정의 하나의 파이프라인으로 묶어주는 Chain 생성|

### Retriever

저장된 벡터 데이터베이스에서 사용자의 질문과 관련된 문서를 검색하는 과정. <br> RAG 시스템의 전반적인 성능과 직결. <br>

<br>

<font style="font-size=20"> 특징 </font>

1\. 정확한 정보 제공: 사용자의 질문과 가장 관련성 높은 정보를 검색. <br>
시스템이 정확하고 유용한 답변을 생성할 수 있도록 함. <br>
이 과정이 효과적으로 이루어지지 않으면, 답변의 품질이 떨어질 수 있음. <br>

2\.응답 시간 단축: 효율적인 검색 알고리즘을 사용하여 데이터베이스에서 적절한 정보를 빠르게 검색함으로써, 전체적인 시스템 응답 시간 단축. <br>
사용자 경험 향상에 직접적인 영향. <br>

3\.최적화: 효과적인 검색 과정을 통해 필요한 정보만을 추출함. <br> 
-> 시스템 자원 사용 최적화, 불필요한 데이터 처리 감소.

<br>

<font style="font-size=20"> 동작 방식 </font>

1\. 질문의 벡터화: 사용자의 질문을 벡터 형태로 변환. <br>
임베딩 단계와 유사한 기술을 사용하여 진행되며, 변환된 질문 벡터는 후속 검색 작업의 기준점으로 사용. <br>

2\.벡터 유사성 비교: 저장된 문서 벡터와 질문 벡터 사이의 유사성을 계산. <br> cosine similarity,, Max Marginal Relevance(MMR) 등의 수학적 방법을 통하여 수행.

3\.상위 문서 선정: 계산된 유사성 점수를 기준으로 상위 N개의 가장 관련성 높은 문서 선정. <br>
이 문서들은 다음 단계에서 사용자의 질문에 대한 답변을 생성하는 데 사용. <br>

4\.문서 정보 반환: 선정된 문서들의 정보를 전달. <br>

<br>

<font style="font-size=20"> Sparse vs Dense </font>

Sparse Retriever와 Dense Retriever는 정보 검색 시스템에서 사용되는 두 가지 주요 방법. <br>
대규모 문서 집합에서 관련 문서를 검색할 때 사용. <br>

<br>

<font style="font-size=18"> Sparse Retriever </font>

문서와 질문을 이산적인 키워드 벡터로 변환하여 처리. <br>
TF-IDF나 BM25와 같은 정보 검색 기법 사용. <br>

- TF-IDF: 단어가 문서에 나타나는 빈도와 그 단어가 몇 개의 문서에서 나타나는지를 반영하여 단어의 중요도 계산. <br>
자주 나타나면서도 문서 집합 전체에서 드물게 나타나는 단어가 높은 가중치를 받음.
- BM25: TF-IDF를 개선한 모델. <br>
문서의 길이를 고려하여 검색 정확도를 향상. <br>
긴 문서와 짧은 문서 간의 가중치를 조정하여, 단어 빈도의 영향을 상대적으로 조절.

<br>

<font style="font-size=18"> Dense Retriever </font>

딥러닝 기법을 사용하여 문서와 쿼리를 연속적인 고차원 벡터로 인코딩. <br>
문서의 의미적 내용을 보다 풍부하게 표현 가능. <br>
키워드가 완벽히 일치하지 않더라도 의미적으로 관련된 문서를 검색 가능. <br>

벡터 공간에서의 거리를 사용하여 쿼리와 가장 관련성 높은 문서를 찾음. <br>
언어의 뉘앙스와 문맥을 이해하는 데 유리하며, 복잡한 쿼리에 대해 더 정확한 검색 결과를 제공 가능. <br>

||Sparse Retriever|Dense Retriever|
|-|----------------|---------------|
|표현 방식|이산적 키워드|연속 벡터|
|의미 처리 능력|문맥과 의미 파악 어려움, 키워드가 정확하게 일치해야 함|문맥과 의미 파악 가능, 키워드가 정확하게 일치하지 않아도 됨|
|적용 범위|간단, 명확한 키워드 검색| 복잡한 질문 등|

#### VectorStore Retriever

vector store를 사용하여 문서를 검색하는 retriever. <br>
similarity search나 MMR과 같은 기법을 통하여 vector store 내 텍스트 쿼리. <br>

<br>

> ```python
> # Retriever를 호출하여 주어진 쿼리에 대한 관련 문서를 반환
> retriever.invoke('tf-idf란 무엇인가?')
> 
> # Configurable
> retriever = db.as_retriever(
>     search_kwargs={'k': 1}
> ).configurable_fields(
>     search_type=ConfigurableField(
>         id='search_type',
>         name='Search Type',
>         description='검색 방법',
>     ),
>     search_kwargs=ConfigurableField(
>         id='search_kwargs',
>         name='Search Kwargs',
>         description='검색 kwargs',
>     ),
> )
> 
> # 상위 세 개 반환
> config = {'configurable': {'search_kwargs': {'k': 3}}}
> retriever.invoke('what embedding is?', config=config)
> 
> # 검색 설정 지정, score_threshold 0.7이상의 문서 반환
> config = {
>     'configurable': {
>         'search_type': 'similarity_score_threshold',
>         'search_kwargs': {
>             'score_threshold': 0.7,
>         },
>     }
> }
> retriever.invoke('what embedding is?', config=config)
> 
> # 검색 설정 지정, mmr 사용.
> config = {
>     'configurable': {
>         'search_type': 'mmr',
>         'search_kwargs': {
>             'k': 2,
>             'fetch_k': 10,
>             'lambda_mult': 0.6,
>         },
>     }
> }
> retriever.invoke('what embedding is?', config=config)
> ```

#### ContextualCompressionRetriever

<img src="https://wikidocs.net/images/page/234097/01-Contextual-Compression.jpeg" width="400">

<br>

사용자의 질문에 대하여 관련성이 높은 정보가 많은 양의 무관한 텍스트를 포함한 문서에 묻혀 있을 수 있을 가능성이 높음. <br>
이러한 전체 문서를 전달하면 LLM 호출에 많은 비용이 들어가며, 낮은 품질의 응답으로 이어질 수 있음. <br>

ContextualCompressionRetriever 검색된 문서를 그대로 반환하는 대신, 주어진 질의의 맥락을 사용하여 문서를 압축함으로써 관련 정보만 반환되도록 함. <br>
- 압축: 개별 문서의 내용을 압축하는 것, 문서를 전체적으로 필터링하는 것

<br>

> ```python
> # EmbeddingsFilter: 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환하여 더 저렴하고 빠른 옵션 제공.
> embeddings_filter = EmbeddingsFilter(
>     embeddings=OpenAIEmbeddings(),
>     similarity_threshold=0.8,   # 유사도가 0.8 이상인 문서 필터
> )
> 
> compression_retriever = ContextualCompressionRetriever(
>     base_compressor=embeddings_filter,
>     base_retriever=retriever,
> )
> 
> compression_retriever.invoke(<question>)
> ```

#### EnsembleRetriever

EnsembleRetriever는 여러 검색기를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능. <br>
다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능을 달성 가능. <br>

<br>

<font style='font-size:20px'> 특징 </font>

1. 여러 검색기 통합: 다양한 유형의 검색기를 입력으로 받아 결과 결합.
2. 결과 재순위화: Reciprocal Rank Fusion 알고리즘을 사용하여 결과의 순위 조정.
3. 하이브리드 검색: 주로 sparse retriever와 dense retriever를 결합하여 사용.
- Sparse retriever: 키워드 기반 검색에 효과적
- Dense retriever: 의미적 유사성 기반 검색에 효과적

<br>

> ```python
> documents: list[str]
> 
> bm25_retriever = BM25Retriever.from_texts(
>     doc_list,
> )
> bm25_retriever.k = 1
> 
> faiss_vectorstore = FAISS.from_texts(
>     doc_list,
>     embedding=OpenAIEmbeddings(model_name='gpt-4o-mini'),
> )
> faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={'k': 1})
> 
> ensemble_retriever = EnsembleRetriever(
>     retrievers=[bm25_retriever, faiss_retriever],
>     weights=[0.7, 0.3],
> )
> 
> query = 'query'
> ensemble_result = ensemble_retriever.invoke(query)
> 
> # configurable
> ensemble_retriever = EnsembleRetriever(
>     retrievers=[bm25_retriever, faiss_retriever],
> ).configurable_fields(
>     weights=ConfigurableField(
>         id='ensemble_weights',
>         name='Ensemble Weights',
>         description='Ensemble Weights',
>     )
> )
> 
> config = {'configurable': {'ensemble_weights': [1, 0]}}
> ensemble_retriever.invoke('what embedding is', config=config)
> ```

#### LongContextReorder

paper: https://arxiv.org/pdf/2307.03172

모델이 긴 컨텍스트 중간에 있는 관련 정보에 접근해야 할 때, 제공된 문서를 무시하는 경향이 있음. <br>
-> 검색 후 문서의 순서를 재배열하여 성능 저하 방지. 

<br>

> ```python
> embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
> texts = [
>     "영화는 다양한 이야기를 시각적으로 표현하는 매체입니다.",
>     "좋은 영화는 관객의 감정을 깊이 있게 자극합니다.",
>     "영화의 배경 음악은 이야기의 분위기를 한층 더 풍부하게 만듭니다.",
>     "고전 영화는 오늘날에도 여전히 많은 사랑을 받고 있습니다.",
>     "상상력이 풍부한 영화는 관객을 새로운 세계로 안내합니다.",
>     "여행은 새로운 경험과 추억을 만들어 줍니다.",
>     "커피 한 잔과 좋은 책은 완벽한 조합입니다.",
>     "친구와의 대화는 언제나 즐거운 시간이 됩니다.",
>     "가을의 바람은 특별한 감성을 불러일으킵니다.",
>     "정원에서 꽃을 가꾸는 것은 마음을 편안하게 합니다."
> ]
> 
> retriever = Chroma.from_texts(
>     texts,
>     embedding=embeddings
>     ).as_retriever(search_kwargs={'k': 10}
> )
> 
> reordering = LongContextReorder()
> documents_reordered = reordering.transform_documents(documents)
> 
> ###################################
> # 사용 예제
> 
> def reorder_documents(documents):
>     reordering = LongContextReorder()
>     reordered_documents = reordering.transform_documents(documents)
>     documents_joined = '\n'.join([document.page_content for document in docs])
> 
>     return documents_joined
> 
> template = '''
> 주어진 context를 활용하라:
> {context}
> 
> 다음 질문에 답하라:
> {question}
> 
> 주어지는 언어로 답변하라: {language}
> '''
> 
> prompt = ChatPromptTemplate.from_template(template)
> model = ChatOpenAI(model='gpt-4o-mini')
> parser = StrOutputParser()
> 
> chain = (
>     {
>         'context': itemgetter('question')
>         | retriever
>         | RunnableLambda(reorder_documents),
>         'question': itemgetter('question'),
>         'language': itemgetter('language'),
>     }
>     | prompt
>     | model
>     | parser
> )
> 
> answer = chain.invoke(
>     {'question': 'ChatGPT에 대해 무엇을 말해줄 수 있나요?', 'language': 'KOREAN'}
> )
> ```

#### ParentDocumentRetriever

문서 검색 과정에서 문서를 적절한 크기의 chunk로 나누는 것은 다음의 상충되는 두 가지 중요한 요소를 고려해야 함.

1. 작은 문서를 원하는 경우: 문서의 임베딩이 그 의미를 가장 정확하게 반영. <br>
문서가 너무 길면 임베딩이 의미를 잃어버릴 수 있음. <br>
2. 각 청크의 맥락이 유지되도록 충분히 긴 문서를 원하는 경우.

위의 두 경우에서 균형을 맞추기 위해 ParentDocumentRetriever라는 도구가 사용. <br> 
문서를 작은 chunk으로 나누고, 이 chunk를 관리. <br>
검색을 진행할 때는, 먼저 이 작은 chunk들을 찾아낸 다음, 이 chunk들이 속한 원본 문서의 ID를 통해 전체적인 맥락을 파악 가능. <br>

parent document: 원본 문서. 전체 문서일 수도 있고, 비교적 큰 다른 chunk일 수도 있음. <br>
이 방식을 통해 문서의 의미를 정확하게 파악하면서도, 전체적인 맥락을 유지할 수 있음 <br>

<br>

> ```python
> retriever = ParentDocumentRetriever(
>     vectorstore=vectorstore,
>     docstore=store,
>     child_splitter=child_splitter,
> )
> 
> parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
> child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
> vectorstore = Chroma(
>     collection_name='split_parents',
>     embedding_function=OpenAIEmbeddings(),
> )
> store = InMemoryStore()
> 
> retriever = ParentDocumentRetriever(
>     vectorstore=vectorstore,
>     docstore=store,
>     child_splitter=child_splitter,
>     parent_splitter=parent_splitter,
> )
> ```

#### SelfQueryRetriever

Self-querying retrivers: [List](https://python.langchain.com/docs/integrations/retrievers/self_query/)

<br>

자체적으로 질문을 생성하고 해결할 수 있는 기능을 갖춘 검색 도구. <br>
이는 사용자가 제공한 자연어 질의를 바탕으로, query-constructing LLM chain을 사용해 구조화된 질의 생성. <br>
이후 생성된 질의를 VectorStore에 적용하여 검색 수행.

이 과정을 통해 사용자의 입력 질의를 저장된 문서의 내용과 의미적으로 비교하는 것을 넘어서, <br>
사용자의 질의에서 문서의 메타데이터에 대한 필터를 추출 하고, 이 필터를 실행하여 관련된 문서를 찾을 수 있음.

<br>

> ```python
> documents: list[Document]
> 
> db = Chroma.from_documents(
>     documents, OpenAIEmbeddings(model='text-embedding-3-small')
> )
> 
> metadata_field_info = [
>     AttributeInfo(
>         name=name1,
>         description=description1,
>         type=type1,
>     ),
>     AttributeInfo(
>         name=name2,
>         description=description2,
>         type=type2,
>     ),
> ]
> 
> model = ChatOpenAI(model='gpt-4o-mini', temperature=0)
> 
> # SelfQueryRetriever 생성
> retriever = SelfQueryRetriever.from_llm(
>     llm=model,
>     vectorstore=db,
>     document_contents=<contents_description>,
>     metadata_field_info=metadata_field_info,
>     enable_limit=True,  # 검색 결과 제한.
>     search_kwargs={'k': 2},  # 검색 결과를 2개로 제한.
> )
> ```

In [16]:
# 문서 목록을 생성합니다.
documents = [
    Document(
        page_content='심혈관 질환에 좋은 오메가3',  # 오메가3가 심혈관 건강에 좋은 내용을 담고 있습니다.
        metadata={'year': 2024, 'category': '영양제', 'user_rating': 4.5}  # 2024년, 영양제 카테고리, 사용자 평점 4.5
    ),
    Document(
        page_content='모든 비타민을 한 번에',  # 여러 비타민을 함께 섭취할 수 있는 제품에 대한 정보입니다.
        metadata={'year': 2024, 'category': '영양제', 'user_rating': 4.2}  # 2024년, 영양제 카테고리, 사용자 평점 4.2
    ),
    Document(
        page_content='피부를 맑고 생기있게',  # 피부 건강과 관련된 화장품에 대한 내용입니다.
        metadata={'year': 2024, 'category': '화장품', 'user_rating': 4.6}  # 2024년, 화장품 카테고리, 사용자 평점 4.6
    ),
    Document(
        page_content='태양으로 부터 피부를 보호하세요',  # 자외선으로부터 피부를 보호하는 제품에 대한 정보입니다.
        metadata={'year': 2023, 'category': '화장품', 'user_rating': 4.7}  # 2023년, 화장품 카테고리, 사용자 평점 4.7
    ),
    Document(
        page_content='비타민d, 맞지 말고 섭취하자',  # 비타민 D 섭취의 중요성을 강조하는 내용입니다.
        metadata={'year': 2024, 'category': '영양제', 'user_rating': 4.6}  # 2024년, 영양제 카테고리, 사용자 평점 4.6
    ),
]

In [10]:
# 메타데이터 필드에 대한 정보를 정의합니다.
metadata_field_info = [
    AttributeInfo(
        name='year',  # 출시 연도를 나타내는 필드
        description='상품 출시 연도',  # 필드 설명
        type='integer',  # 데이터 타입은 정수
    ),
    AttributeInfo(
        name='category',  # 상품 카테고리를 나타내는 필드
        description='상품의 품목 구분. ["영양제", "화장품"]으로 구성',  # 필드 설명
        type='string',  # 데이터 타입은 문자열
    ),
    AttributeInfo(
        name='user_rating',  # 사용자 평점을 나타내는 필드
        description='제품에 대한 유저의 평균 점수. 1 ~ 5점으로 구성',  # 필드 설명
        type='float',  # 데이터 타입은 실수
    ),
]

In [17]:
# SelfQueryRetriever를 초기화하여 LLM과 벡터 저장소를 연결합니다.
retriever = SelfQueryRetriever.from_llm(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    
    vectorstore=Chroma.from_documents(
        documents,  # 앞서 정의한 문서 목록을 벡터 저장소에 추가
        OpenAIEmbeddings(model='text-embedding-3-small')  # 텍스트 임베딩을 위한 모델 설정
    ),
    
    document_contents='간략한 상품 정보',  # 문서의 내용 요약을 나타내는 설명
    metadata_field_info=metadata_field_info,  # 메타데이터 필드 정보
)

In [23]:
retriever.invoke('user_rating이 4.5이상인 카테고리가 영양제인 상품을 추천해줘')

[Document(metadata={'category': '영양제', 'user_rating': 4.6, 'year': 2024}, page_content='비타민d, 맞지 말고 섭취하자'),
 Document(metadata={'category': '영양제', 'user_rating': 4.6, 'year': 2024}, page_content='비타민d, 맞지 말고 섭취하자'),
 Document(metadata={'category': '영양제', 'user_rating': 4.5, 'year': 2024}, page_content='심혈관 질환에 좋은 오메가3'),
 Document(metadata={'category': '영양제', 'user_rating': 4.5, 'year': 2024}, page_content='심혈관 질환에 좋은 오메가3')]

#### TimeWeightedVectorStoreRetriever

의미론적 유사성과 시간에 따른 감쇠를 결합해 사용하는 검색 도구. <br>
문서 또는 데이터의 신선함과 관련성을 모두 고려하여 결과 제공. <br>
객체가 마지막으로 접근된 시간을 기준으로 하여 정보의 신선함 평가. <br>
-> 자주 접근되는 객체는 시간이 지나도 높은 점수를 유지. <br>
-> 자주 사용되거나 중요하게 여겨지는 정보가 검색 결과 상위에 위치할 가능성이 높음. <br>
-> 최신성과 관련성을 모두 고려하는 동적인 검색 결과 제공.

<br>

score: $ \text{semantic similarity} + (1.0 - \text{decay rate})^{\text{passed hour}} $
- semantic similarity: 문서 또는 데이터 간의 의미적 유사도
- decay rate: 시간이 지남에 따라 점수가 얼마나 감소하는지 나타내는 비율
- passed hour: 객체가 마지막으로 접근된 후부터 현재까지 경과한 시간 단위

<br>

> ```python
> retriever = TimeWeightedVectorStoreRetriever(
>     vectorstore=db,
>     decay_rate=0.001, # 작으면 감쇠율이 낮아 정보를 잃지 않고, 높으면 감쇠율이 높아 정보를 빨리 잃음.
>     k=1,
> )
> 
> retriever.add_documents(
>     [
>         Document(
>             page_content='금요일이다~',
>             metadata={'last_accessed_at': datetime(2024, 10, 24)},
>         ),
>         Document(
>             page_content='아싸 금요일이다~',
>         ),
>     ]
> )
> 
> retriever.invoke('금요일~')
> ```

In [11]:
# FAISS 벡터 저장소를 초기화합니다.
db = FAISS(
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-small'),  # 텍스트 임베딩을 위한 모델 설정
    index=faiss.IndexFlatL2(1536),  # L2 거리 기반의 FAISS 인덱스를 생성 (1536차원)
    docstore=InMemoryDocstore(),  # 메모리 내 문서 저장소
    index_to_docstore_id={},  # 인덱스와 문서 저장소 ID 간의 매핑 (초기화는 비어있음)
)

# 시간 가중치 벡터 저장소 리트리버를 초기화합니다.
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=db,  # 위에서 생성한 FAISS 벡터 저장소를 사용
    decay_rate=0.999,  # 시간에 따라 가중치를 감소시키는 비율 설정
    k=1,  # 반환할 가장 유사한 문서의 개수
)

In [12]:
# 리트리버에 문서를 추가합니다.
retriever.add_documents(
    [
        Document(
            page_content='월요일이네...',  # 문서 내용
            metadata={'last_accessed_at': datetime(2024, 10, 25)}  # 문서의 마지막 접근 날짜 메타데이터
        ),
        Document(
            page_content='아씨 월요일이네...',  # 또 다른 문서 내용
            # 이 문서에는 메타데이터가 설정되지 않음
        ),
    ]
)

['80b70ec8-af81-4230-9b9a-938df47aa795',
 'a889c85a-b423-4529-a20c-07fe01e4a025']

In [13]:
retriever.invoke('월요일...')

[Document(metadata={'last_accessed_at': datetime.datetime(2024, 10, 28, 10, 22, 28, 730855), 'created_at': datetime.datetime(2024, 10, 28, 10, 22, 26, 221641), 'buffer_idx': 1}, page_content='아씨 월요일이네...')]

In [54]:
nest_asyncio.apply()

In [5]:
def get_page_urls(url):
    # 주어진 URL에 GET 요청을 보냅니다.
    response = requests.get(url)
    
    # 응답의 HTML 내용을 lxml 파서를 사용하여 BeautifulSoup 객체로 변환합니다.
    bs_response = BeautifulSoup(response.text, 'lxml')
    
    # 'a.nclicks(cnt_flashart)' 선택자로 링크를 찾아 href 속성을 추출합니다.
    urls = [
        item.get('href')
        for item in bs_response.select('a.nclicks\(cnt_flashart\)')
    ]

    # 추출한 URL 리스트를 반환합니다.
    return urls

## TimeWeightedVectorStoreRetriever (Colab)
# 1. 네이버 뉴스에서 조선일보 10.24 ~ 10.28의 기사 수집
# 10.28 <- url

data = {}
# 빈 리스트를 초기화하여 URL을 저장할 준비를 합니다.
for date in range(24, 26):
    urls = []

    # tqdm을 사용하여 진행 상황을 시각적으로 보여줍니다.
    for oid in tqdm(('023',)):
        # 각 oid에 대해 1페이지부터 15페이지까지 반복합니다.
        for page in range(1, 15):
            # 각 페이지의 URL을 생성합니다.
            url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date=202410{date}&page={page}'
            
            # 생성된 URL에서 페이지 URL을 추출하고 urls 리스트에 추가합니다.
            urls.extend(get_page_urls(url))

    # 중복된 URL을 제거하여 고유한 URL 리스트로 만듭니다.
    urls = list(set(urls))

    data.update(
        {datetime(2024, 10, date): naver_news_crawler(urls, request_per_second=10)}
    )


# documents 내의 개별 document 내부의  metadata: source 밖에 없음
# document의 metadata last_accessed_at은
# 2. 수집된 기사를 document로 만듦 (metadata에 last_accessed_at은 기사 일자)
#    가지고 온 후에 last_accessed_at을 삽입
# 데이터를 기반으로 각 문서의 메타데이터를 업데이트합니다.
for key, value in data.items():
    for document in value:
        document.metadata.update({'last_accessed_at': key})  # 마지막 접근 날짜를 메타데이터에 추가

# 텍스트를 일정 크기로 나누기 위한 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 각 청크의 최대 크기
    chunk_overlap=50,  # 청크 간의 중첩 크기
)

# 데이터의 값을 가져와서 하나의 리스트로 결합
temp = data.values()
documents = list(itertools.chain(*temp))  # 중첩된 리스트를 평탄화
documents = text_splitter.split_documents(documents)  # 문서를 청크로 분할

# 3. FAISS 벡터 저장소 구축 (GPU 버전)
db = FAISS(
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-small'),  # 임베딩 함수 설정
    index=faiss.IndexFlatL2(1536),  # L2 거리 기반 인덱스 설정 (1536차원)
    docstore=InMemoryDocstore(),  # 메모리 내 문서 저장소
    index_to_docstore_id={}  # 인덱스와 문서 저장소 ID 간의 매핑 (초기화는 비어있음)
)

# 4. 시간 가중치 벡터 저장소 리트리버 생성
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=db,  # 앞서 생성한 FAISS 벡터 저장소를 사용
    decay_rate=0.9,  # 시간에 따른 가중치 감소 비율 설정
    k=1,  # 반환할 가장 유사한 문서의 개수
)

# 5. 분할된 문서를 리트리버에 추가
retriever.add_documents(documents)

# 6. 조회 수행
retriever.invoke('')  # 빈 문자열로 조회 실행

Fetching pages: 100%|##########| 217/217 [00:08<00:00, 26.11it/s]


In [38]:
retriever.invoke('삼성전자의 근황에 대해서 알려줘')

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/023/0003865904', 'last_accessed_at': datetime.datetime(2024, 10, 28, 14, 1, 53, 356316), 'created_at': datetime.datetime(2024, 10, 28, 12, 57, 21, 315171), 'buffer_idx': 108}, page_content='최근 외국투자기관들이 한국 경제 피크론, 한국 대표 기업에 대한 부정적 전망 보고서를 잇따라 내고 있다. 외국인투자자들은 한국 대표기업 삼성전자에 대해 30일 거래일 연속 순매도 행진을 이어가고 있다. /그래픽=조선디자인랩 김영재\t\t\t\t\t\t\t\t\t\t한국은행이 법인세 신고 대상인 기업 93만여 개의 지난해 실적을 분석한 결과, 매출이 전년 대비 1.5% 감소해 2010년 통계 작성 후 최악이었다. 코로나 팬데믹 때인 2020년의 마이너스 1.1%보다도 낮았다. 영업 이익률은 3년 연속 하락세를 거듭하며 매출액 대비 3.5%로 떨어져')]

### Chain

모듈(chain)을 연결해 다양한 기능 구현. <br>

#### Generic Chain

##### LLMChain

사용자 입력을 기반으로 prompt template으로 prompt를 생성해 LLM 호출

<br>

> ```python
> # 단일 변수
> template = '''
> Question: {question}
> 
> Answer: 
> '''
> 
> prompt = PromptTemplate(
>     input_variables=['question'],
>     template=template,
> )
> 
> chain = LLMChain(
>     llm=ChatOpenAI(model_name='gpt-4o-mini'),
>     prompt=prompt,
> )
> 
> chain.predict(question=<question>)
>
> # 다중 변수
> template = '''
> Question: {question}
> 
> Language: {language}
>
> Answer: 
> '''
> 
> prompt = PromptTemplate(
>     input_variables=['question', 'language'],
>     template=template,
> )
> 
> chain = LLMChain(
>     llm=ChatOpenAI(model_name='gpt-4o-mini'),
>     prompt=prompt,
> )
> 
> chain.predict(question=<question>, language=<language>)
> ```

In [29]:
# 템플릿을 정의합니다. 질문과 답변 형식을 설정합니다.
template = '''
question: {question}

answer: 
'''

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate(
    template=template,  # 위에서 정의한 템플릿 사용
    input_variables=['question'],  # 입력 변수로 'question'을 설정
)

# LLM 체인을 생성하여 언어 모델과 프롬프트를 연결합니다.
chain = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    prompt=prompt,  # 정의한 프롬프트 템플릿 사용
)

C:\Users\USER\AppData\Local\Temp\ipykernel_14352\3427926968.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [30]:
chain.invoke('삼성전자의 근황에 대해서 알려줄래?')

{'question': '삼성전자의 근황에 대해서 알려줄래?',
 'text': '삼성전자는 최근 몇 가지 주요 이슈와 성과가 있습니다. 2023년에는 반도체 시장의 회복세와 함께 메모리 반도체 수요가 증가하고 있으며, 이를 통해 수익성이 개선되고 있는 상황입니다. 또한, 삼성전자는 인공지능(AI), 5G, 전장(자동차 전자기기) 분야에서도 활발한 연구와 투자를 진행하고 있습니다.\n\n특히, 삼성전자는 스마트폰 사업 부문에서도 갤럭시 Z 폴드와 Z 플립 시리즈와 같은 폴더블 스마트폰의 성공적인 출시로 주목받고 있습니다. 이러한 혁신적인 제품들은 소비자들 사이에서 긍정적인 반응을 얻고 있으며, 고급 스마트폰 시장에서의 경쟁력을 강화하고 있습니다.\n\n또한, 친환경 경영과 지속 가능성에 대한 노력도 계속하고 있으며, 재생 가능 에너지 사용 확대와 제품의 친환경 설계를 추진하고 있습니다. 이러한 다양한 분야에서의 활동을 통해 삼성전자는 글로벌 기술 기업으로서의 입지를 더욱 확고히 하고 있습니다.'}

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


In [31]:
# 템플릿을 정의합니다. 질문과 언어, 그리고 답변 형식을 설정합니다.
template = '''
question: {question}

language: {language}

answer: 
'''

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate(
    template=template,  # 위에서 정의한 템플릿 사용
    input_variables=['question', 'language'],  # 입력 변수로 'question'과 'language' 설정
)

# LLM 체인을 생성하여 언어 모델과 프롬프트를 연결합니다.
chain = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    prompt=prompt,  # 정의한 프롬프트 템플릿 사용
)

In [32]:
chain.invoke({'question': '삼성전자의 근황에 대해서 알려줘', 'language': '일본어'})

{'question': '삼성전자의 근황에 대해서 알려줘',
 'language': '일본어',
 'text': '最近の三星電子（サムスン電子）に関するニュースは、主に以下のポイントに焦点を当てています。\n\n1. **半導体産業**: サムスンは、半導体市場での競争力を維持するために大規模な投資を行っています。特に、次世代のメモリチップやプロセッサの開発に注力しています。\n\n2. **スマートフォン市場**: サムスンは、最新のGalaxyシリーズのスマートフォンを発表し、折りたたみ式スマートフォンの分野でも革新を続けています。特に、Z FoldやZ Flipシリーズは人気を集めています。\n\n3. **環境への取り組み**: サムスンは持続可能な製品開発を進めており、リサイクル可能な材料の使用やエネルギー効率の向上に力を入れています。\n\n4. **グローバル展開**: 世界各国でのビジネス展開を強化しており、特に米国や欧州市場での存在感を高めています。\n\n5. **AIと5G技術**: AI（人工知能）や5G（第5世代移動通信システム）技術の研究開発にも積極的に取り組んでおり、これらの分野でのリーダーシップを目指しています。\n\nこれらの取り組みを通じて、サムスンは競争力を維持し、さらなる成長を目指しています。'}

##### SimpleSequentialChain

입출력이 하나씩 있는 여러 개의 체인을 연결.

<br>

> ```python
> # prompt1
> template1 = '''
> 너는 특정 주제에 대해 칼럼을 작성하는 비평가이다. 주제가 주어졌을 때 이것에 관련하여 글을 작성하라.
> 
> 주제: {subject}
> 내용: 
> '''
> prompt1 = PromptTemplate(
>     input_variables=['subject'],
>     template=template1,
> )
> 
> chain1 = LLMChain(
>     llm=ChatOpenAI(model_name='gpt-4o-mini'),
>     prompt=prompt1,
> )
> 
> # prompt2
> template2 = '''
> 당신은 비평에 대해 반론을 작성하는 작가이다. 특정 글이 주어졌을 때 이를 반박하는 글을 작성하라.
> 
> 칼럼: {column}
> 반박글: 
> '''
> prompt2 = PromptTemplate(
>     input_variables=['column'],
>     template=template2
> )
> 
> chain2 = LLMChain(
>     llm=ChatOpenAI(model_name='gpt-4o-mini'),
>     prompt=prompt2,
> )
> 
> chain = SimpleSequentialChain(
>     chains=[chain1, chain2],
> )
> chain.invoke(<subject>)
> ```

In [47]:
# 첫 번째 템플릿을 정의합니다. 주어진 산업 분야에 대해 기업을 임의로 선택하는 봇의 기능을 설정합니다.
template1 = '''
너는 특정 종목에 대하여 기업을 분석하는 봇이다.
산업 분야가 주어질 때 해당 산업 분야에서 하나의 기업을 임의로 선택하여 알려줘.
답변은 기업 명만 알려줘.

산업: {industry}

내용: 
'''

# 프롬프트 템플릿 생성
prompt1 = PromptTemplate(
    template=template1,  # 위에서 정의한 템플릿 사용
    input_variables=['industry'],  # 입력 변수로 'industry' 설정
)

# LLM 체인 생성
chain1 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    prompt=prompt1,  # 정의한 프롬프트 템플릿 사용
)

# 산업 분야 '금융'에 대해 chain1을 실행
chain1.invoke({'industry': '금융'})


# 두 번째 템플릿을 정의합니다. 주어진 회사에 대한 정보를 요약하는 봇의 기능을 설정합니다.
template2 = '''
너는 기업의 정보를 요약하여 보여주는 봇이다. 특정 회사가 주어질 때, 이 회사에 대해 정보를 알려줘.

회사: {company}
회사 정보: 
'''

# 프롬프트 템플릿 생성
prompt2 = PromptTemplate(
    template=template2,  # 위에서 정의한 템플릿 사용
    input_variables=['company'],  # 입력 변수로 'company' 설정
)

# LLM 체인 생성
chain2 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    prompt=prompt2  # 정의한 프롬프트 템플릿 사용
)

# 두 체인을 순차적으로 연결하는 간단한 시퀀셜 체인 생성
chain = SimpleSequentialChain(
    chains=[chain1, chain2],  # chain1과 chain2를 포함
)

In [51]:
print(chain.invoke('첨단 반도체 제조업').get('output'))

**회사: 삼성전자**

**설립 연도**: 1969년

**본사**: 대한민국 수원시

**산업 분야**: 전자기기, 반도체, 모바일, 가전제품 등

**주요 제품**:
- 스마트폰 (갤럭시 시리즈)
- 반도체 (DRAM, NAND 플래시 메모리)
- TV (QLED, OLED)
- 가전제품 (냉장고, 세탁기, 에어컨 등)
- 웨어러블 기기 (갤럭시 워치)

**시장 위치**: 
삼성전자는 세계 최대의 반도체 제조업체이자 스마트폰 시장에서 주요한 점유율을 가진 기업으로, 글로벌 전자기기 시장에서 중요한 역할을 하고 있습니다.

**연구개발**: 
매년 막대한 연구개발 비용을 투자하여 혁신적인 기술을 개발하고 있으며, 인공지능, 5G, IoT 등의 분야에서도 활발히 활동하고 있습니다.

**사회적 책임**: 
환경 지속 가능성 및 사회 공헌 활동에도 적극적이며, 다양한 재활용 프로그램과 사회적 프로젝트를 운영하고 있습니다.

**재무 정보**: 
2022년 기준으로, 삼성전자의 연간 매출은 약 240조 원에 달하며, 지속적인 성장세를 보이고 있습니다.

**기타 정보**: 
삼성전자는 '삼성 갤럭시' 브랜드를 통해 소비자에게 다양한 모바일 기기를 제공하며, 글로벌 시장에서의 경쟁력을 유지하고 있습니다.


##### SequentialChain

복수 개의 입출력을 가진 체인 연결. <br>

입출력이 하나씩 있는 여러 개의 체인을 연결.

<br>

> ```python
> # prompt1
> template1 = '''
> 너는 특정 주제에 대해 칼럼을 작성하는 비평가이다. 주제와 연도가 주어졌을 때 이것에 관련하여 글을 작성하라.
> 
> 연도: {year}
> 주제: {subject}
> 내용: 
> '''
> prompt1 = PromptTemplate(
>     input_variables=['year', 'subject'],
>     template=template1,
> )
> 
> chain1 = LLMChain(
>     llm=ChatOpenAI(model_name='gpt-4o-mini'),
>     prompt=prompt1,
>     output_key='column',
> )
> 
> # prompt2
> template2 = '''
> 당신은 비평에 대해 반론을 작성하는 작가이다. 특정 글이 주어졌을 때 이를 반박하는 글을 작성하라.
> 
> 칼럼: {column}
> 반박글: 
> '''
> prompt2 = PromptTemplate(
>     input_variables=['column'],
>     template=template2,
> )
> 
> chain2 = LLMChain(
>     llm=ChatOpenAI(model_name='gpt-4o-mini'),
>     prompt=prompt2,
>     output_key='counterargument',
> )
> 
> chain = SimpleSequentialChain(
>     chains=[chain1, chain2],
>     input_variables=['year', 'subject'],
>     output_variables=['column', 'counterargument'],
> )
> chain.invoke({'yaer': <year>, 'subject': <subject>})
> ```

In [56]:
# 첫 번째 템플릿을 정의합니다. 주어진 산업 분야와 국가에 대해 기업을 임의로 선택하는 봇의 기능을 설정합니다.
template1 = '''
너는 특정 종목에 대하여 기업을 분석하는 봇이다.
산업 분야가 주어질 때 해당 산업 분야에서 하나의 기업을 임의로 선택하여 알려줘.
답변은 기업 명만 알려줘.

산업: {industry}

국가: {country}

내용: 
'''

# 프롬프트 템플릿 생성
prompt1 = PromptTemplate(
    template=template1,  # 위에서 정의한 템플릿 사용
    input_variables=['industry', 'country'],  # 입력 변수로 'industry'와 'country' 설정
)

# LLM 체인 생성
chain1 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    prompt=prompt1,  # 정의한 프롬프트 템플릿 사용
    output_key='company',  # 출력 키 설정 (선택된 기업 이름)
)

# 두 번째 템플릿을 정의합니다. 주어진 회사에 대한 정보를 요약하는 봇의 기능을 설정합니다.
template2 = '''
너는 기업의 정보를 요약하여 보여주는 봇이다. 특정 회사가 주어질 때, 이 회사에 대해 정보를 알려줘.

회사: {company}

회사 정보: 
'''

# 프롬프트 템플릿 생성
prompt2 = PromptTemplate(
    template=template2,  # 위에서 정의한 템플릿 사용
    input_variables=['company'],  # 입력 변수로 'company' 설정
)

# LLM 체인 생성
chain2 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    prompt=prompt2,  # 정의한 프롬프트 템플릿 사용
    output_key='summarization',  # 출력 키 설정 (회사의 정보 요약)
)

# 두 체인을 순차적으로 연결하는 시퀀셜 체인 생성
chain = SequentialChain(
    chains=[chain1, chain2],  # chain1과 chain2를 포함
    input_variables=['industry', 'country'],  # 입력 변수 설정
    output_variables=['company', 'summarization'],  # 출력 변수 설정
)

In [57]:
chain.invoke({'industry': '첨단 반도체 제조업', 'country': '대만'})

{'industry': '첨단 반도체 제조업',
 'country': '대만',
 'company': 'TSMC (Taiwan Semiconductor Manufacturing Company)',
 'summarization': '**회사명:** TSMC (Taiwan Semiconductor Manufacturing Company)\n\n**설립 연도:** 1987년\n\n**본사:** 대만 신주\n\n**업종:** 반도체 제조\n\n**주요 사업:** TSMC는 세계 최대의 반도체 파운드리(위탁 생산) 기업으로, 다양한 종류의 반도체 칩을 설계한 고객사에 맞춰 제조 서비스를 제공합니다. 주요 고객으로는 애플, AMD, NVIDIA, 퀄컴 등 여러 글로벌 기술 기업이 있습니다.\n\n**기술력:** TSMC는 첨단 반도체 제조 공정에서 선두주자로, 5nm, 7nm, 10nm 등 최신 기술을 보유하고 있으며, 3nm 공정도 개발 중입니다. 이러한 공정 기술은 스마트폰, 컴퓨터, 데이터 센터 및 IoT 기기 등 다양한 전자 제품에 사용됩니다.\n\n**시장 위치:** TSMC는 전세계 반도체 파운드리 시장에서 약 55% 이상의 점유율을 차지하고 있으며, 반도체 산업에서 중요한 역할을 하고 있습니다.\n\n**연구 개발:** TSMC는 지속적인 연구 개발에 투자하고 있으며, 새로운 반도체 기술과 공정 개발을 위한 혁신을 추진하고 있습니다.\n\n**최근 동향:** TSMC는 대만 외에도 미국, 일본 등지에 공장을 설립하고 있으며, 글로벌 반도체 공급망의 안정성을 강화하기 위한 노력을 기울이고 있습니다.\n\n**재무 성과:** TSMC는 꾸준한 매출 성장을 기록하고 있으며, 반도체 산업의 성장과 함께 안정적인 수익성을 유지하고 있습니다.\n\n이와 같은 정보는 TSMC의 기업 전략과 시장 내 위치를 이해하는 데 도움을 줍니다.'}

In [64]:
# 첫 번째 템플릿을 정의합니다. 고객 서비스 문의를 분류하는 봇의 기능을 설정합니다.
template1 = '''
고객 서비스 문의를 입력으로 받는다.
각 문의를 기본 범주와 세부 범주로 분류한다.
json 형식으로 답변을 출력하고, 키는 primary와 secondary로 분류한다.

기본 범주: 주문 취소, 배송 조회

주문 취소 세부 범주:
- 환불 문의
- 교환 문의
- 반품 문의

배송 조회 세부 범주:
- 운송장 확인 문의
- 실시간 위치 문의
- 기타 문의

고객 문의: {question}
답변: 
'''

# 프롬프트 템플릿 생성
prompt1 = PromptTemplate(
    template=template1,  # 위에서 정의한 템플릿 사용
    input_variables=['question'],  # 입력 변수로 'question' 설정
)

# LLM 체인 생성
chain1 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    prompt=prompt1,  # 정의한 프롬프트 템플릿 사용
    output_key='category'  # 출력 키 설정 (분류된 카테고리)
)

# 두 번째 템플릿을 정의합니다. 배송 조회 맥락에서 문제 해결을 위한 봇의 기능을 설정합니다.
template2 = '''
고객 서비스 문의를 입력으로 받는다.
배송 조회 맥락에서 문제 해결이 필요한 경우 사용자를 도와줄래?
고객이 겪는 문제에 대한 구체적인 카테고리는 다음과 같아:
{category}

위의 카테고리가 아래의 선택지와 유사하다면 아래 선택지에서 가장 적절한 선택지를 하나를 골라서 고객에게 답해줘.
- 실시간 배송 조회가 필요한 경우 조회를 할 수 있는 링크 제공
- 상담원 연결을 통한 배송 조회를 진행하고 싶은 경우 02-1234-5678 제공
- 사용자가 이 주제와 관련 없는 질문을 시작 시, "해당 주제에 대한 답을 드릴 수 없습니다. 죄송합니다"
  라는 문구를 띄우며, 채팅을 종료하길 원하는지 확인. 확인 후 요청을 다음 분류체계에 따라 분류.
  > primary, secondary key로 분류하며, json 형식으로 출력
'''

# 프롬프트 템플릿 생성
prompt2 = PromptTemplate(
    template=template2,  # 위에서 정의한 템플릿 사용
    input_variables=['category']  # 입력 변수로 'category' 설정
)

# LLM 체인 생성
chain2 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    prompt=prompt2,  # 정의한 프롬프트 템플릿 사용
    output_key='answer'  # 출력 키 설정 (고객에게 제공할 답변)
)

# 두 체인을 순차적으로 연결하는 시퀀셜 체인 생성
chain = SequentialChain(
    chains=[chain1, chain2],  # chain1과 chain2를 포함
    input_variables=['question'],  # 입력 변수 설정
    output_variables=['category', 'answer'],  # 출력 변수 설정
)

In [65]:
print(chain.invoke('주문한 물건이 아직 도착 안 했는데 물건의 현재 위치를 알고 싶어').get('answer'))

고객님의 요청에 따라 실시간 배송 조회를 도와드리겠습니다. 배송의 실시간 위치를 확인하시려면 아래 링크를 클릭하여 조회해 주시기 바랍니다.

[실시간 배송 조회 링크](#)

추가적인 도움이 필요하시면 언제든지 말씀해 주세요!


#### Index Chain

공개되지 않은 고유 데이터를 이용해 질의응답을 하기 위한 체인. <br>

##### RetrievalQA

retriever를 통해 질의응답. <br>

<br>

> ```python
> chain = RetrievalQA.from_chain_type(
>     llm=ChatOpenAI(model_name='gpt-4o-mini'),
>     chain_type='stuff',
>     retriever=retriever,
> )
> 
> chain.invoke(<question>)
> ```

<br>

chain_type:
- stuff: 모든 관련 데이터를 context로 prompt에 담아 언어 모델에 전달
- map_reduce: 관련 데이터를 chunk로 분할하여 chunk별로 llm을 호출하고 마지막으로 모든 결과를 결합하는 prompt로 llm을 호출
- refine: 관련 데이터를 chunk로 분할하여 첫 번째 chunk마다 prompt를 생성해서 llm을 호출하고, 그 출력과 함꼐 다음 chunk에서 prompt를 생성해서 llm을 호출을 반복.
- map_rerank: 관련 데이터를 chunk로 분할하여 chunk별로 llm을 호출하여 그 답변이 얼마나 확실한지를 나타내는 점수를 표시. <br>
이 점수에 따라 응답으 순위를 매려 가장 높은 응답의 순위를 매겨 가장 높은 점수를 받은 응답을 반환.

In [4]:
news = '''
도널드 트럼프 전 미국 대통령이 '반도체 보조금'을 강하게 비판하면서 현지 투자를 결정한 삼성전자와 SK하이닉스의 불안감이 커지고 있다. 약 일주일 앞으로 다가온 미 대선에서 트럼프 전 대통령이 당선될 경우 우리 기업 타격이 우려된다.

27일(현지시간) 뉴욕타임스(NYT)에 따르면 공화당 미 대선 후보인 트럼프 전 대통령은 최근 팟캐스트 인터뷰에서 반도체과학법(이하 반도체법)을 두고 "정말 나쁜 거래"라고 말했다.

바이든 행정부에서 시행된 반도체법은 미국에 투자하는 반도체 기업에 보조금을 주는 것이 핵심 내용이다. 트럼프 전 대통령은 "(미국 정부가) 10센트도 낼 필요가 없었다"며 반도체 수입 시 높은 관세를 매기면 보조금을 안 줘도 반도체 기업이 알아서 미국에 공장을 건설할 것이라고 주장했다.

이런 발언은 트럼프 전 대통령이 재선에 성공해도 반도체 보조금에 큰 변화가 없을 것이란 일각의 전망과 대비된다. 산업연구원은 최근 '미국 대선 시나리오별 한국 산업 영향과 대응 방향' 보고서에서 "(트럼프 당선 시) 반도체법 기획·입안 시기가 트럼프 1기였다는 점을 고려할 때 국가전략상 이 법에 의한 보조금·세액공제 혜택 축소 가능성은 낮을 것으로 예상된다"고 했다.

트럼프 전 대통령이 인터뷰에서 '대만'을 직접 언급한 점에 비춰볼 때 TSMC를 주요 표적으로 삼은 것으로 보인다. 그러나 트럼프 전 대통령이 대통령에 당선돼 실제로 보조금을 축소·폐지할 경우 삼성전자, SK하이닉스 역시 영향이 불가피하다.
'''

In [8]:
# 주어진 뉴스 내용을 줄 단위로 나누어 빈 줄을 제외한 리스트를 생성합니다.
texts = [sentence for sentence in news.split('\n') if sentence]  # news를 '\n'으로 분할하고, 빈 문자열을 제외하여 리스트 생성

In [19]:
# FAISS 데이터베이스를 텍스트 목록으로부터 생성합니다.
db = FAISS.from_texts(
    texts=texts,  # 이전에 생성한 텍스트 리스트
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),  # 텍스트 임베딩을 위한 모델 설정
    metadatas=[  # 각 텍스트에 대한 메타데이터
        {'source': 1},
        {'source': 2},
        {'source': 3},
        {'source': 4},
        {'source': 5},
    ],
    ids=[1, 2, 3, 4, 5]  # 각 텍스트에 대한 고유 ID
)

# 생성한 FAISS 데이터베이스를 리트리버로 변환합니다.
retriever = db.as_retriever()

In [12]:
# RetrievalQA 체인을 생성합니다.
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    chain_type='stuff',  # 체인 유형 설정 (여기서는 'stuff' 타입 사용)
    retriever=retriever,  # 이전에 설정한 리트리버 사용
)

In [13]:
chain.invoke('미국 투자에서 이슈사항이 뭐야?')

{'query': '미국 투자에서 이슈사항이 뭐야?',
 'result': '미국 투자에서 현재 이슈가 되고 있는 사항은 도널드 트럼프 전 대통령의 반도체 보조금 비판입니다. 트럼프 전 대통령이 당선될 경우, 삼성전자와 SK하이닉스와 같은 반도체 기업들이 받을 보조금이 축소되거나 폐지될 우려가 커지고 있습니다. 이는 바이든 행정부에서 시행된 반도체법에 따라 미국에 투자하는 반도체 기업에 보조금을 주는 정책과 관련이 있습니다. 트럼프 전 대통령은 보조금 없이도 높은 관세를 부과하면 기업들이 자발적으로 미국에 공장을 설립할 것이라고 주장하고 있습니다. 이러한 상황은 한국 기업들에게 상당한 영향을 미칠 수 있습니다.'}

##### RetrievalQAWithSourcesChain

retriever를 통해 질의응답. <br>
source가 포함된 응답 결과 출력. <br>

<br>

> ```python
> chain = RetrievalQAWithSourcesChain.from_chain_type(
>     llm=ChatOpenAI(model_name='gpt-4o-mini'),
>     chain_type='stuff',
>     retriever=retriever,
> )
> 
> chain.invoke(<question>)
> ```


In [20]:
# RetrievalQAWithSourcesChain 체인을 생성합니다.
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    chain_type='map_reduce',  # 체인 유형 설정 (여기서는 'map_reduce' 타입 사용)
    retriever=retriever,  # 이전에 설정한 리트리버 사용
)

In [21]:
chain.invoke('미국 투자에서 이슈사항이 뭐야?')

{'question': '미국 투자에서 이슈사항이 뭐야?',
 'answer': '미국 투자에서 이슈사항은 도널드 트럼프 전 대통령이 반도체 보조금을 비판하면서 삼성전자와 SK하이닉스의 불안감이 커지고 있다는 점입니다. 또한, 트럼프 전 대통령이 당선될 경우 한국 기업에 타격이 우려되고 있으며, 그의 정책이 반도체 법안에 영향을 미칠 가능성이 있다는 점도 이슈로 제기되고 있습니다.\n\n',
 'sources': '1, 3, 5, 4'}

##### SummarizeChain

document 요약. 

<br>

> ```python
> chain = load_summarize_chain(
>   llm=ChatOpenAI(model_name='gpt-4o-mini')
>   chain_type='map_reduce',
> 
> )
> 
> chain.invoke(<documents>)
> ```

In [22]:
# 요약 체인을 로드합니다.
chain = load_summarize_chain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    chain_type='map_reduce',  # 체인 유형 설정 (여기서는 'map_reduce' 타입 사용)
)

In [24]:
nest_asyncio.apply()

In [94]:
# Naver 뉴스 크롤러를 사용하여 주어진 URL에서 뉴스 기사를 가져옵니다.
documents = naver_news_crawler(['https://n.news.naver.com/mnews/article/008/0005106377'])

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  6.25it/s]


In [95]:
# 텍스트를 분할하기 위한 RecursiveCharacterTextSplitter를 생성합니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 각 청크의 최대 크기 설정 (300자)
    chunk_overlap=50,  # 청크 간의 중첩 크기 설정 (50자)
)

# 문서를 지정한 크기로 분할합니다.
documents = text_splitter.split_documents(documents)

# 분할된 문서 리스트를 출력합니다.
documents

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/008/0005106377'}, page_content='"보조금 안주고 관세로 공장 유치?"…불안한 삼성전자·SK하이닉스\n\n\n\n입력2024.10.28. 오후 3:02\n\n\n수정2024.10.28. 오후 3:03\n\n기사원문\n \n\n\n\n\n유선일 기자\n\n\n\n\n\n\n\n\n\n\n유선일 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇'),
 Document(metadata={'source': 'https://n.news.naver.com/mnews/article/008/0005106377'}, page_content='분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게'),
 Document(metadata={'source': 'https://n.news.naver.com/mnews/article/008/0005106377'}, p

In [96]:
chain(documents)

C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\langchain_openai\chat_models\base.py:369: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


{'input_documents': [Document(metadata={'source': 'https://n.news.naver.com/mnews/article/008/0005106377'}, page_content='"보조금 안주고 관세로 공장 유치?"…불안한 삼성전자·SK하이닉스\n\n\n\n입력2024.10.28. 오후 3:02\n\n\n수정2024.10.28. 오후 3:03\n\n기사원문\n \n\n\n\n\n유선일 기자\n\n\n\n\n\n\n\n\n\n\n유선일 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇'),
  Document(metadata={'source': 'https://n.news.naver.com/mnews/article/008/0005106377'}, page_content='분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게'),
  Document(metadata={'source': 'https://n.news.naver.com/mnews/artic

In [ ]:
documents = list(itertools.chain(*data.values()))

In [20]:
for document in documents:
    document.metadata.update({'범주': '정치'})

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
)

In [13]:
documents = text_splitter.split_documents(documents)

In [78]:
# FAISS 데이터베이스를 문서 목록으로부터 생성합니다.
db = FAISS.from_documents(
    documents,  # 사용할 문서 리스트
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),  # 텍스트 임베딩을 위한 모델 설정
)

# 생성한 FAISS 데이터베이스를 리트리버로 변환합니다.
retriever = db.as_retriever(search_kwargs={'k': 10})  # 상위 10개의 유사 문서를 검색하도록 설정

# 리트리버를 사용하는 메모리 객체를 생성합니다.
memory = VectorStoreRetrieverMemory(retriever=retriever, input_key='category')  # 'category'를 입력 키로 설정

In [79]:
# 뉴스 데이터
# 상황: 유저는 극작가이며, 최근 일어나는 사회 현상을 바탕으로 시나리오를 작성하려고한다.
# 
# prompt1: 아래 범주에서 유저 원하는 시나리오에 부합하는 유사한 범주를 선택
# 범주: 경제, 정치, 사회, 생활/문화, IT/과학, 세계
template1 = '''
유저로부터 작성하고 싶은 시나리오를 입력받는다. 
입력받은 대략적인 시나리오를 아래의 범주에서 하나 선택하여 반환.

범주:
- 경제, 정치, 사회, 생활/문화, IT/과학, 세계

시나리오: {question}
답변: 
'''
prompt1 = PromptTemplate(
    template=template1,
    input_variables=['question'],
)

chain1 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),
    prompt=prompt1,
    output_key='category'
)

# prompt2: 입력받은 범주에 해당하는 데이터를 db에서 조회 (유사도가 높은 상위 10개) (RetrievalQA)
template2 = '''
범주를 입력받아 해당 범주에 속하는 기사 반환

{category}

답변: 
'''
prompt2 = PromptTemplate(
    template=template2,
    input_variables=['question', 'category'],
)

chain2 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),
    prompt=prompt2,
    memory=memory,
    output_key='references'
)

 
# prompt3: 조회된 데이터를 바탕으로 시나리오 작성
template3 = '''
유저로부터 작성하고 싶은 시나리오를 입력받는다. 
입력받은 기사를 참고하여 작성 시나리오 작성 .

기사:
{references}

지시사항:
{question}

답변: 
'''
prompt3 = PromptTemplate(
    template=template3,
    input_variables=['question', 'references', 'history'],
)

chain3 = LLMChain(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),
    prompt=prompt3,
    output_key='scenario'
)

In [83]:
# 여러 체인을 순차적으로 실행하는 SequentialChain을 설정합니다.
chain = SequentialChain(
    chains=[chain1, chain2, chain3],  # 실행할 체인 목록 (chain1, chain2, chain3)
    input_variables=['question'],  # 첫 번째 체인에 입력으로 사용할 변수 설정
    output_variables=['category', 'references', 'scenario'], 
)


In [84]:
chain.invoke({'question': '경제사범이 대통령이 되는 이야기'})

{'question': '경제사범이 대통령이 되는 이야기',
 'category': '범주: 정치',
 'references': '정치 범주에 속하는 기사는 다음과 같습니다:\n\n1. **대선 후보들, 정책 공약 발표**  \n   각 정당의 대선 후보들이 주요 정책 공약을 발표하며 유권자들의 관심을 끌고 있습니다. 특히, 경제, 교육, 복지 분야에서의 차별화된 정책이 주목받고 있습니다.\n\n2. **국회, 새로운 법안 통과**  \n   국회에서 통과된 새로운 법안이 사회적 논란을 일으키고 있습니다. 이번 법안은 환경 보호와 관련된 내용이 포함되어 있으며, 찬반 의견이 갈리고 있습니다.\n\n3. **여야, 세제 개편 논의**  \n   여당과 야당이 세제 개편에 대한 논의를 시작했습니다. 이번 개편안은 중산층과 저소득층을 지원하는 방향으로 수립될 예정입니다.\n\n4. **정치인을 향한 여론 조사 결과 발표**  \n   최근 실시된 여론 조사에서 특정 정치인의 지지율이 상승하고 있는 것으로 나타났습니다. 이에 대한 각 당의 반응이 주목받고 있습니다.\n\n5. **국제 정치 상황에 대한 분석**  \n   최근 국제 정치 상황이 복잡해지면서, 한국의 외교 정책에 대한 새로운 접근이 필요하다는 목소리가 커지고 있습니다. 전문가들은 다자간 협력의 중요성을 강조하고 있습니다.\n\n이 외에도 정치 관련 다양한 뉴스가 있으며, 관심 있는 주제를 더 깊이 탐구해보시는 것을 추천드립니다.',
 'scenario': '**시나리오 제목: "정치의 반전: 경제사범이 대통령이 되다"**\n\n**장르**: 드라마 / 정치 스릴러\n\n**배경**: 현대 한국, 대선이 다가오는 시점. 경제와 복지에 대한 논의가 뜨거운 가운데, 예상치 못한 정치적 사건이 발생한다.\n\n**등장인물**:\n1. **김민수** (경제사범 출신): 과거 대기업의 경제범죄로 기소된 후, 복역을 마치고 사회로 복귀. 이제는 경제 개혁과 서민을 위한 정책을 주장하며 정치에 발을 들인다.\n2. 

### Agent

![](https://brightinventions.pl/static/98578848ff94ebbf2fd58883fbe3e780/d9199/llm_agents_loop.png)

사용자의 요청에 따라 어떤 기능을 어떤 순서대로 실행할지 결정하는 모듈. <br>
chain은 미리 정해진 기능을 수행하나, agent는 사용자의 요청에 따라 수행하는 기능이 달라짐. <br>
agent가 수행하는 특정 기능을 tool이라고 함. <br>

<br>

<font style="font-size=18"> 동작 순서 </font>

1. 입력: agent에게 작업 부여
2. 행동: 사용할 tool과 tool에 대한 입력 결정
3. 관찰: tool의 출력 결과 관찰
4. 추론: 무엇을 해야할지 생각
5. 완료: 작업 완료를 판단할 때까지 반복

#### Tool

agent에서 복잡한 작업을 수행하기 위한 도구. <br>
LLM에게 지식과 계산 능력 부여 등의 역할. <br>

<br>

agent: 
- zero-shot-react-description: 도구에 대한 설명만으로 사용할 도구 결정
- conversational-react-description: 이전 대화 내용 기억

##### SerpApi

site: https://serpapi.com/

계산 지식 엔진.

<br>

> ```python
> tools = load_tools(['serpapi'])
> 
> memory = ConversationBufferMemory(
>     memory_key='chat_history',
>     return_messages=True,
> )
> 
> agent = initialize_agent(
>     agent='conversational-react-description',
>     llm=ChatOpenAI(model='gpt-4o-mini'),
>     tools=tools,
>     memory=memory
> )
> 
> agent.invoke(<question>)
> ```


In [36]:
# 필요한 도구를 로드합니다. 여기서는 SERP API를 사용합니다.
tools = load_tools(['serpapi'])

# 대화의 기억을 관리하기 위한 메모리를 설정합니다.
memory = ConversationBufferMemory(
    memory_key='chat_history',  # 대화 기록을 저장할 키 설정
    return_messages=True,  # 메시지를 반환하도록 설정
)

# 대화형 에이전트를 초기화합니다.
agent = initialize_agent(
    agent='conversational-react-description',  # 사용할 에이전트 유형 설정
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    tools=tools,  # 로드한 도구들을 에이전트에 전달
    memory=memory,  # 설정한 메모리를 에이전트에 전달
)

In [38]:
agent.invoke('삼성전자가 사과문을 작성한 이유를 웹에서 찾아볼래?')

{'input': '삼성전자가 사과문을 작성한 이유를 웹에서 찾아볼래?',
 'chat_history': [HumanMessage(content='현재 삼성전자의 주가를 웹에서 검색해서 알려줄래?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='현재 삼성전자의 주가는 58,100 KRW입니다. 이전 종가는 55,900 KRW였습니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='삼성전자가 사과문을 작성한 이유를 웹에서 찾아볼래?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='삼성전자가 사과문을 작성한 이유는 여러 가지가 있습니다. 최근 삼성전자의 3분기 영업이익이 시장의 기대를 크게 밑돌면서 경영진이 사과문을 발표했습니다. 이는 삼성 반도체 부문에 대한 위기설을 해소하기 위한 노력으로 해석됩니다. 또한, 이재용 부회장은 경영권 승계와 관련된 위법 논란에 대해 사과하며, 삼성 그룹 내 노조 활동을 보장하겠다는 의지를 표명했습니다. 더불어, 삼성은 과거의 경영 문제와 사회적 소통 부족에 대한 반성과 함께, 법과 윤리를 엄격히 준수하겠다는 다짐을 포함한 사과문을 발표했습니다.', additional_kwargs={}, response_metadata={})],
 'output': '삼성전자가 사과문을 작성한 이유는 여러 가지가 있습니다. 최근 삼성전자의 3분기 영업이익이 시장의 기대를 크게 밑돌면서 경영진이 사과문을 발표했습니다. 이는 삼성 반도체 부문에 대한 위기설을 해소하기 위한 노력으로 해석됩니다. 또한, 이재용 부회장은 경영권 승계와 관련된 위법 논란에 대해 사과하며, 삼성 그룹 내 노조 활동을 보장하겠다는 의지를 표명했습니다. 더불어, 삼성은 과거의 경영 문제와 사회적 소통 부족에 대한 반성과 함께, 법과 윤리를 엄격히 준수하겠다

##### Wolfram Alpha

site: https://www.wolframalpha.com

계산 지식 엔진.

<br>

> ```python
> tools = load_tools(['wolfram-alpha'])
> 
> memory = ConversationBufferMemory(
>     memory_key='chat_history',
>     return_messages=True,
> )
> 
> agent = initialize_agent(
>     agent='zero-shot-react-description',
>     llm=ChatOpenAI(model='gpt-4o-mini'),
>     tools=tools,
>     memory=memory
> )
> 
> agent.invoke(<question>)
> ```


In [42]:
# Wolfram Alpha 도구를 로드합니다.
tools = load_tools(['wolfram-alpha'])

# 대화의 기억을 관리하기 위한 메모리를 설정합니다.
memory = ConversationBufferMemory(
    memory_key='chat_history',  # 대화 기록을 저장할 키 설정
    return_messages=True,  # 메시지를 반환하도록 설정
)

# 에이전트를 초기화합니다.
agent = initialize_agent(
    agent='zero-shot-react-description',  # 사용할 에이전트 유형 설정 (Zero-shot 반응 설명)
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    tools=tools,  # 로드한 도구들을 에이전트에 전달
    memory=memory,  # 설정한 메모리를 에이전트에 전달
)

In [43]:
agent.invoke('서울과 부산의 거리가 몇 km야?')

{'input': '서울과 부산의 거리가 몇 km야?',
 'chat_history': [HumanMessage(content='서울과 부산의 거리가 몇 km야?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='서울과 부산의 거리는 약 328.1 km입니다.', additional_kwargs={}, response_metadata={})],
 'output': '서울과 부산의 거리는 약 328.1 km입니다.'}

In [49]:
# Google Serper 도구를 로드합니다.
tools = load_tools(['google-serper'])

# 대화의 기억을 관리하기 위한 메모리를 설정합니다.
memory = ConversationBufferMemory(
    memory_key='chat_history',  # 대화 기록을 저장할 키 설정
    return_messages=True,  # 메시지를 반환하도록 설정
)

# 에이전트를 초기화합니다.
agent = initialize_agent(
    agent='zero-shot-react-description',  # 사용할 에이전트 유형 설정 (Zero-shot 반응 설명)
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    tools=tools,  # 로드한 도구들을 에이전트에 전달
    memory=memory,  # 설정한 메모리를 에이전트에 전달
)

In [52]:
agent.invoke('삼성전자의 위기가 어떤 위기야?')

{'input': '삼성전자의 위기가 어떤 위기야?',
 'chat_history': [HumanMessage(content='이번 주 한국의 날씨에 대해서 알려줄래?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Agent stopped due to iteration limit or time limit.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='삼성전자의 위기가 어떤 위기야?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='삼성전자의 위기는 2023년 반도체 불황으로 인한 매출 및 영업이익 감소와 함께, 주요 사업 분야에서의 경쟁 격화, 한국 제조업의 전반적인 위기와 산업 공동화 우려와 관련이 있다.', additional_kwargs={}, response_metadata={})],
 'output': '삼성전자의 위기는 2023년 반도체 불황으로 인한 매출 및 영업이익 감소와 함께, 주요 사업 분야에서의 경쟁 격화, 한국 제조업의 전반적인 위기와 산업 공동화 우려와 관련이 있다.'}

### RAGAS

reference: https://docs.ragas.io/en/stable/

합성 테스트 데이터셋 생성 라이브러리. <br>

In [3]:
def get_page_urls(url):
    # 주어진 URL에 GET 요청을 보냅니다.
    response = requests.get(url)
    
    # 응답의 HTML 내용을 lxml 파서를 사용하여 BeautifulSoup 객체로 변환합니다.
    bs_response = BeautifulSoup(response.text, 'lxml')
    
    # 'a.nclicks(cnt_flashart)' 선택자로 링크를 찾아 href 속성을 추출합니다.
    urls = [
        item.get('href')
        for item in bs_response.select('a.nclicks\(cnt_flashart\)')
    ]

    # 추출한 URL 리스트를 반환합니다.
    return urls

In [5]:
data = {}
# 빈 리스트를 초기화하여 URL을 저장할 준비를 합니다.
for date in range(24, 25):
    urls = []

    # tqdm을 사용하여 진행 상황을 시각적으로 보여줍니다.
    for oid in tqdm(('023',)):
        # 각 oid에 대해 1페이지부터 15페이지까지 반복합니다.
        for page in range(1, 15):
            # 각 페이지의 URL을 생성합니다.
            url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date=202410{date}&page={page}'
            
            # 생성된 URL에서 페이지 URL을 추출하고 urls 리스트에 추가합니다.
            urls.extend(get_page_urls(url))

    # 중복된 URL을 제거하여 고유한 URL 리스트로 만듭니다.
    urls = list(set(urls))

    data.update(
        {datetime(2024, 10, date): naver_news_crawler(urls, request_per_second=5)}
    )

Fetching pages: 100%|##########| 277/277 [00:10<00:00, 26.75it/s]


#### Generate

reference: https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/

<br>

> ```python
> generator_llm = LangchainLLMWrapper(ChatOpenAI(model_name='gpt-4o-mini'))
> generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model='text-embedding-small-3'))
> 
> trans = default_transforms(
>     llm=transformer_llm,
>     embedding_model=generator_embeddings,
> )
> 
> # default ratio
> query_distribution = default_query_distribution(generator_llm)
> 
> # 비율 임의 조정
> query_distribution = [
>     (AbstractQuerySynthesizer(llm=generator_llm), 0.25),
>     (ComparativeAbstractQuerySynthesizer(llm=generator_llm), 0.25),
>     (SpecificQuerySynthesizer(llm=generator_llm), 0.5),
> ]
> 
> generator = TestsetGenerator(llm=generator_llm)
> dataset = generator.generate_with_langchain_docs(
>     documents,
>     testset_size=1,
>     transforms=trans,
>     query_distribution=query_distribution,
> )
> dataset.to_frame()
> ```

In [ ]:
# 주어진 날짜에 해당하는 문서를 데이터에서 가져옵니다.
documents = data.get(datetime(2024, 10, 24, 0, 0))  # 2024년 10월 24일의 문서를 검색

In [ ]:
# LLM을 래핑하여 LangchainLLMWrapper 생성
generator_llm = LangchainLLMWrapper(ChatOpenAI(model_name='gpt-4o-mini'))  # GPT-4o-mini 모델을 래핑

# 임베딩 모델을 래핑하여 LangchainEmbeddingsWrapper 생성
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())  # OpenAI 임베딩 모델을 래핑

# 기본 변환을 설정합니다.
transform = default_transforms(
    llm=generator_llm,  # 래핑된 LLM 전달
    embedding_model=generator_embeddings,  # 래핑된 임베딩 모델 전달
)

# 기본 쿼리 분포를 설정합니다.
query_distribution = default_query_distribution(generator_llm)  # 래핑된 LLM을 기반으로 쿼리 분포 생성

# 테스트셋 생성기를 초기화합니다.
generator = TestsetGenerator(llm=generator_llm)  # 래핑된 LLM을 사용하여 테스트셋 생성기 설정

In [ ]:
# 주어진 문서로부터 테스트셋을 생성합니다.
dataset = generator.generate_with_langchain_docs(
    documents,  # 테스트셋을 생성할 문서 리스트
    testset_size=10,  # 생성할 테스트셋의 크기 설정 (10개)
    transforms=transform,  # 적용할 변환 설정
    query_distribution=query_distribution,  # 쿼리 분포 설정
)

Applying [SummaryExtractor, HeadlinesExtractor]:   0%|          | 0/80 [00:00<?, ?it/s]

Prompt fix_output_format failed to parse output: The output parser failed to parse the output after 0 retries.
Prompt headlines_extractor_prompt failed to parse output: The output parser failed to parse the output after 0 retries.
unable to apply transformation: The output parser failed to parse the output after 0 retries.


Applying EmbeddingExtractor:   0%|          | 0/40 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/40 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node


...

Applying SummaryCosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating common_concepts:   0%|          | 0/1 [00:00<?, ?it/s]

Generating common themes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [70]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What ideas come from not having a clear theme ...,"[\n, \n]",Not having a clear theme in a context can lead...,AbstractQuerySynthesizer
1,What concepual implicatons arise from the abse...,"[\n, \n]",The absence of a defined theme in a given cont...,AbstractQuerySynthesizer
2,What conceptual implicashuns arise from the ab...,"[\n, \n]",The absence of a defined theme in a given cont...,AbstractQuerySynthesizer
3,How do reports show 'apartment' culture in S. ...,[The concept of 'apartment' in South Korea has...,Reports show that 'apartment' culture in South...,ComparativeAbstractQuerySynthesizer
4,How do North Korean troops' involvement in Ukr...,[The concept of 'apartment' in South Korea has...,The involvement of North Korean troops in Ukra...,ComparativeAbstractQuerySynthesizer
5,"How do reports on health care, K-culture, Nort...",[The concept of 'apartment' in South Korea has...,"Reports on health care, K-culture, North Korea...",ComparativeAbstractQuerySynthesizer
6,What role do wearable robots play in the expan...,[웨어러블 로봇은 대표적인 서비스 로봇으로 분류된다. 현재 로봇 산업은 공장에서 사...,Wearable robots are classified as representati...,SpecificQuerySynthesizer
7,What factors are being considered in the revie...,[尹대통령 “북한군 활동 따라 우크라 살상무기 지원 검토”\n\n\n\n입력2024...,President Yoon's statement indicates that the ...,SpecificQuerySynthesizer
8,What implications does the activity of North K...,[尹대통령 “북한군 활동 따라 우크라 살상무기 지원 검토”\n\n\n\n입력2024...,The activity of North Korean forces has implic...,SpecificQuerySynthesizer
9,What inspired 신애라 to adopt 예은이?,[막내 예진 양과의 만남도 떠올렸다. 신애라는 “셋째는 18살이다. 생후 100일 ...,신애라는 예은이를 입양하면서 자매를 만들어 주는 것이 가장 큰 선물이라고 생각했다.,SpecificQuerySynthesizer


#### Evalute

reference: https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/

<br>

>```python
> evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model_name='gpt-4o-mini'))
> evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
> 
> metrics = [
>     LLMContextRecall(llm=evaluator_llm), 
>     FactualCorrectness(llm=evaluator_llm), 
>     Faithfulness(llm=evaluator_llm),
>     SemanticSimilarity(embeddings=evaluator_embeddings)
> ]
> 
> results = evaluate(
>     dataset=Dataset.from_pandas(dataset.to_pandas().rename(columns={'reference_contexts': 'retrieved_contexts'})),
>     metrics=metrics,
> )
> ```

<br>

Metric:
- context_recall: 얼마나 많은 관련 reference가 성공적으로 검색되었는지 측정.
- factual_correctness: 생성된 response가 reference와 얼마나 일치하는지 측정.
- faithfulness: 생성된 답변의 사실적 일관성을 주어진 컨텍스트와 비교하여 측정하는 지표
- semantic_simialrity: 

In [86]:
# LLM을 래핑하여 LangchainLLMWrapper 생성
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model_name='gpt-4o-mini'))  # GPT-4o-mini 모델을 래핑

# 임베딩 모델을 래핑하여 LangchainEmbeddingsWrapper 생성
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())  # OpenAI 임베딩 모델을 래핑

# 평가 지표를 설정합니다.
metrics = [
    LLMContextRecall(llm=evaluator_llm),  # LLMContextRecall 메트릭을 사용하여 평가 지표 리스트 생성
]

# 데이터셋을 평가합니다.
evaluate(
    dataset=Dataset.from_pandas(dataset.to_pandas().rename(columns={'reference_contexts': 'retrieved_contexts'})),  # 데이터셋 변환 및 열 이름 변경
    metrics=metrics,  # 설정한 평가 지표 전달
)

Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\pydantic\main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `dict[str, any]` but got `EvaluationResult` with value `{'context_recall': 0.6818}` - serialized value may not be as expected
  Expected `dict[str, any]` but got `EvaluationResult` with value `{'context_recall': 0.6818}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'context_recall': 0.6818}